# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は機械翻訳にニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は機械翻訳を行っていただきます。

ハンズオンについての説明
http://qiita.com/GushiSnow/private/b0abf7d3dccafe14fa07

機械翻訳は機械が言語を別の言語に翻訳するものです。

機械翻訳にはいくつか種類があるのでここでも紹介しておきます。

* PBMT(Phrase Base Machine Translation)モデル
 * [moses](http://www.statmt.org/moses/)というオープンソースで使用できるメジャーな機械翻訳のモデルですが、難しすぎて理解できない人を続出させる機械翻訳の鬼門です
* ニューラル機械翻訳
 * 翻訳元単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで翻訳先言語を学習させる手法

以下では、このChainerを利用しデータを準備するところから実際にNN翻訳モデルを構築し学習・評価を行うまでの手順を解説します。

1. [各種ライブラリ導入](#各種ライブラリ導入) 
2. [機械翻訳のクラス](#機械翻訳のクラス)
3. [翻訳処理を行うforward処理部分](#翻訳処理を行うforward処理部分)
4. [各値を設定](#各値を設定)
5. [実行](#実行)



## 1.各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。



In [1]:
from util.functions import trace
import numpy as np

from util.chainer_cpu_wrapper import wrapper

from EncoderDecoderModel import EncoderDecoderModel

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入
* `util`:今回の処理で必要なライブラリが入っています。


## 2.機械翻訳のクラス

下記を設定しています。
* ニューラルネットを用いて機械翻訳用のモデルを構成しています。

全体構成

![](./pictures/NN_machine_translation.png)




## 3.翻訳処理を行うforward処理部分

下記の論文を参考にしてforward処理を記述しています。

http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf

* Encoder部分
もっとも特徴的な部分は翻訳元言語を逆順にしていることです。そうすることで精度が向上していると述べており、今回の翻訳のNNモデルもそれを再現しています。

この論文でははっきりした要因はわかっていないが、おそらく翻訳先の言語と翻訳元言語の距離が逆順にすることで最初の単語の距離が近くなり、翻訳のタイムラグが少なくなったことが起因していると考えられています。

![](./pictures/encoder.png)

* Decoder部分

学習部分と予測部分を実装しています。学習部分ではターゲット先の単語の取得と損失の計算をしています。
またlstmで次回の学習に使用する部分では学習では正解の翻訳、予測では予測した翻訳を使用しています。

![](./pictures/decorder.png)

In [3]:
class EncoderDecoderModelForward(EncoderDecoderModel):

    def __init__(self):
        pass
    
    def __forward(self, is_training, src_batch, trg_batch = None, generation_limit = None):
        m = self.__model
        tanh = functions.tanh
        lstm = functions.lstm
        batch_size = len(src_batch)
        src_len = len(src_batch[0])
        src_stoi = self.__src_vocab.stoi
        trg_stoi = self.__trg_vocab.stoi
        trg_itos = self.__trg_vocab.itos
        s_c = wrapper.zeros((batch_size, self.__n_hidden))

#--------Hands on------------------------------------------------------------------#
    # encoding
        """
　　     逆順にEncodingしていくため下記の手順になる。 
　　     １：翻訳元言語の末尾</s>を潜在空間に写像し、隠れ層に入力、lstmで出力までをバッチサイズ分行う。
　　     ２：翻訳元言語を逆順に１と同様の処理を行う。
　　     ３：次のミニバッチ処理のために最終結果をlstmで出力。翻訳の仮説用のリストを保持
        """
        s_x = wrapper.make_var([src_stoi('</s>') for _ in range(batch_size)], dtype=np.int32)
        s_i = tanh(m.w_xi(s_x))
        s_c, s_p = lstm(s_c, m.w_ip(s_i))

        for l in reversed(range(src_len)):
            s_x = wrapper.make_var([src_stoi(src_batch[k][l]) for k in range(batch_size)], dtype=np.int32)
            s_i = tanh(m.w_xi(s_x))
            s_c, s_p = lstm(s_c, m.w_ip(s_i) + m.w_pp(s_p))

        s_c, s_q = lstm(s_c, m.w_pq(s_p))
        hyp_batch = [[] for _ in range(batch_size)]

    # decoding
        """
　　     学習と予測をコードしている。
　　     学習部分 
　　     １：学習では損失の計算及び答えとなる翻訳先言語の長さを考慮する必要がある。（翻訳元言語と翻訳先言語で長さが異なるため）
　　     ２：ニューラルネットの処理は基本的にEncodingと同一であるが、損失計算と翻訳仮説候補の確保の処理が加わっている。
        """
        """
        予測部分
        １：予測では予測する翻訳言語の長さに制約をしないとニューラル翻訳モデルの性質上、無限に翻訳してしまう可能性があるので、長さに制約を設けている。
        ２：翻訳元言語を逆順に１と同様の処理を行う。
        ３：基本的に学習部分と同一であるが、ミニバッチサイズ分の翻訳仮説の末尾が</s>になったときにDecoding処理が終わるようになっている。
        """
        if is_training:
            accum_loss = wrapper.zeros(())
            trg_len = len(trg_batch[0])

            for l in range(trg_len):
                s_j = tanh(m.w_qj(s_q))
                r_y = m.w_jy(s_j)
                s_t = wrapper.make_var([trg_stoi(trg_batch[k][l]) for k in range(batch_size)], dtype=np.int32)
                accum_loss += functions.softmax_cross_entropy(r_y, s_t)
                output = wrapper.get_data(r_y).argmax(1)

                for k in range(batch_size):
                    hyp_batch[k].append(trg_itos(output[k]))

                s_c, s_q = lstm(s_c, m.w_yq(s_t) + m.w_qq(s_q))

            return hyp_batch, accum_loss
        else:
            while len(hyp_batch[0]) < generation_limit:
                s_j = tanh(m.w_qj(s_q))
                r_y = m.w_jy(s_j)
                output = wrapper.get_data(r_y).argmax(1)

                for k in range(batch_size):
                    hyp_batch[k].append(trg_itos(output[k]))

                if all(hyp_batch[k][-1] == '</s>' for k in range(batch_size)): break

                s_y = wrapper.make_var(output, dtype=np.int32)
                s_c, s_q = lstm(s_c, m.w_yq(s_y) + m.w_qq(s_q))

            return hyp_batch

#--------Hands on------------------------------------------------------------------#

        
    def train(self, src_batch, trg_batch):
        self.__opt.zero_grads()
        hyp_batch, accum_loss = self.__forward(True, src_batch, trg_batch=trg_batch)
        accum_loss.backward()
        self.__opt.clip_grads(10)
        self.__opt.update()
        return hyp_batch

    def predict(self, src_batch, generation_limit):
        return self.__forward(False, src_batch, generation_limit=generation_limit)

## 4.各値を設定

各値を設定

* モードを学習かテストか設定
* 翻訳元言語の設定
* 翻訳先言語の設定
* 語彙の設定
* 潜在空間の設定
* 隠れ層の設定
* 学習回数の設定
* ミニバッチサイズの設定
* 最大予測言語数の設定


In [4]:
parameter_dict = {}
mode = "train"
#--------Hands on  2----------------------------------------------------------------#
parameter_dict["source"] = "source_wakati_kytea.txt"
parameter_dict["target"] = "target_stanford_parse.txt"
parameter_dict["vocab"] = 500
parameter_dict["embed"] = 20
parameter_dict["hidden"] = 20
parameter_dict["epoch"] = 20
parameter_dict["minibatch"] = 64
parameter_dict["generation_limit"] = 256
parameter_dict["show_hands_on_number"] = 10
parameter_dict["show_i_epoch"] = 0
#--------Hands on  2----------------------------------------------------------------#

## 5.実行

In [5]:
def main():

    trace('initializing ...')
    wrapper.init()

    encoderDecoderModel = EncoderDecoderModel(parameter_dict)
    #encoderDecoderModel.load_data("source_wakati_kytea.txt")
    if mode == 'train': encoderDecoderModel.train_model()
    elif mode == 'test': encoderDecoderModel.test_model()


if __name__ == '__main__':
    main()

2015-12-03 20:19:25.269353 ... initializing ...
2015-12-03 20:19:25.270116 ... making vocaburaries ...
2015-12-03 20:19:25.575983 ... making model ...
2015-12-03 20:19:25.585326 ... epoch 1/20: 
2015-12-03 20:19:25.748562 ... epoch   1/ 20, sample        1
2015-12-03 20:19:25.749047 ...   src = 索引 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2015-12-03 20:19:25.749999 ...   trg = Index * *
2015-12-03 20:19:25.750816 ...   hyp = tuple database integers
2015-12-03 20:19:25.751228 ... epoch   1/ 20, sample        2
2015-12-03 20:19:25.751670 ...   src = 目次 * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
2015-12-03 20:19:25.752363 ...   trg = Contents * *
2015-12-03 20:19:25.753070 ...   hyp = tuple data